In [9]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import math

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn import preprocessing as pre

import csv
import plotly.graph_objects as go
import plotly.express as px
import statistics
from scipy import stats

from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import random

In [2]:
def XY_hosp(df):
    p1=df.hosp_patients.values.tolist()

    df2=df.iloc[0:df.shape[0],]
    df2['hosp_patients_shift']=p1
    

    return df2

In [3]:
def XY_ICU(df):
    p1=df.icu_patients.values.tolist()

    df2=df.iloc[0:df.shape[0],]
    df2['icu_patients_shift']=p1
    

    return df2

In [4]:
def shift_data(df, var, shift_step, timstep):
    p1=var[shift_step:].values.tolist()
    for i in range(shift_step):
        p1.append(0)
    #p1.append(0)
    #p1.append(0)
    #p1.append(0)
    #p1.append(0)
    
    df2=df.iloc[0:df.shape[0],]
    df2['shifted_outcomes']=p1
    
    X_new_hosp=df.iloc[-(shift_step+timstep-1):,:]

    return df2,X_new_hosp

In [5]:
# need to transform the 26 weeks into 6 week per set
def create_sw(X_train,Y_train, train_size, model_inputsize,var_num, list1, list2, kernel, c, eps, gamma, test_size):
 
    
    X_SVR = []
    Y_SVR =[]
    #print(len(X_train))
    for i in range(train_size):
        xset=X_train[i*var_num:i*var_num+var_num*model_inputsize]
        X_SVR.append(xset)
    
    for j in range(train_size):
        yset=Y_train[j:j+4]
       # print(yset)
        Y_SVR.append(yset.ravel())
        #print(yset.ravel())
    Y_SVR=np.array(Y_SVR)
    
    X_SVRtrain=X_SVR[0:train_size-1]
    X_SVRtest=[X_SVR[train_size-1]]
    Y_SVRtrain=Y_SVR[0:train_size-1]
    Y_SVRtest=[Y_SVR[train_size- 1]]
    #print(X_SVRtest)
    #print(Y_SVRtrain)
    
    test_predictions=swsvr( kernel,c,eps,gamma,X_SVRtrain, Y_SVRtrain, X_SVRtest,  test_size)
    return test_predictions



In [6]:
def swsvr( kernel, C_best, eps_best, gamma_best, X_SVRtrain, Y_SVRtrain, X_SVRtest,  Xtest_size):
    svr = MultiOutputRegressor(SVR(kernel=kernel, C=C_best,epsilon=eps_best, gamma=gamma_best))
    #print(Y_SVRtrain)
    #print(X_SVRtrain)
    #print(len(Y_SVRtrain))
    svr.fit(X_SVRtrain,Y_SVRtrain)

    predictions = []
    for i in range(Xtest_size):
        y_pred = svr.predict(X_SVRtest[i].reshape(1,-1))
        predictions.append(y_pred)
    predictions=np.array(predictions).reshape(-1,1)
    return predictions #, contribution_score



In [7]:
def traininng_sliding_window(scaler,Xt,Yt, train_size, model_inputsize,forecast_steps, list1, list2, num_var,kernel,C, eps, gamma, X_testsize):

    X = []
    Y = []
    ymean=[]
    predictions=[]
    for i in range(Xt.shape[0]-train_size-model_inputsize+1):
        sc_X=scaler.fit_transform(Xt.iloc[i:i+ train_size+model_inputsize-1,0].values.reshape(-1,1))

        for j in range(len(list1)):
            sc=scaler.fit_transform(Xt.iloc[i:i + train_size+model_inputsize-1,j+1].values.reshape(-1,1))
            sc_X=np.concatenate((sc_X, sc), axis=1)
        
        for k in range(len(list2)):
            sc_X=np.concatenate((sc_X, Xt.iloc[i:i + train_size+model_inputsize-1,k+len(list1)+1].values.reshape(-1,1)), axis=1)
        X.append(sc_X.flatten())
        #print(len(sc_X.flatten()))
        y=Yt.iloc[(i+model_inputsize-1):(i+train_size+model_inputsize+forecast_steps-1-1), 0]
        #print(y)
        ymean=y.mean()
        ysd=y.std()
        y_sc=scaler.fit_transform(y.values.reshape(-1,1))
        Y.append(y_sc)
        #print(sc_X.flatten())
        #print(len(sc_X.flatten()))
   # for i in range(len(X)):
        pred=create_sw(sc_X.flatten(),y_sc,train_size,model_inputsize,num_var, list1, list2, kernel, C, eps, gamma,X_testsize)
        predictions_original=pred*ysd+ymean
        predictions.append(predictions_original)
 
    return predictions

In [8]:

def rmse(actual, prediction):
    rmse_value =sqrt(np.square(np.subtract(actual,prediction)).mean())
    return rmse_value

def mape(actual,prediction):
    mape_value = np.mean(np.abs((actual - prediction)/actual))
    return mape_value

def MAE_test(actual, prediction):
    mae_value=np.mean(np.abs(actual - predictions))
    return mae_value